In [1]:

from utils import csv_to_dict, show_images, show_images_single, bb_intersection_over_union
import pandas as pd
import numpy as np
import json
from collections import defaultdict

In [2]:
# Process CSV file and convert to dictionary (key by image path) and will gather bounding boxes
json_path = '../data/test_split.json'
#csv_data = pd.read_csv(train_csv_path)
#dict_data = csv_to_dict(csv_data=csv_data)
with open(json_path, 'r') as file:
    dict_data = json.load(file)

# Process CSV file and convert to dictionary (key by image path) and will gather bounding boxes
result_csv_path = '../data/evaluation.csv'
csv_data = pd.read_csv(result_csv_path)
result_dict_data = csv_to_dict(csv_data=csv_data)

In [3]:
images_to_print = defaultdict(list)
groundTruth_mage = defaultdict(list)
mislabeled, bad_iou, unpaired, total_bb = 0,0,0,0
all_problematic_bb = []
accurate_predictions = defaultdict(list)
accurate_ground_truth = defaultdict(list)

for key in dict_data.keys():
    ground_truth_bb = []
    ground_truth_centroids = []
    model_output_bb = []
    model_output_centroids = []
    problematic_bb = []
  
    for label_dict in dict_data[key]:
        #ground truth boxes
        xmax, xmin, ymax, ymin = [label_dict.get(key) for key in ['xmax', 'xmin', 'ymax', 'ymin']]
        total_bb+=1
        ground_truth_bb.append([xmin, ymin, xmax, ymax])
        #print(xmax, xmin, ymax, ymin)

    
    for label_dict in result_dict_data[key]:
        #model result boxes
        xmax, xmin, ymax, ymin = [label_dict.get(key) for key in ['xmax', 'xmin', 'ymax', 'ymin']]     
        model_output_bb.append([xmin, ymin, xmax, ymax])

    num_gt_bbox = len(ground_truth_bb)
    num_mo_bbox = len(model_output_bb)
    iou_table = np.zeros((num_gt_bbox, num_mo_bbox))

    for i, bb_ground_truth in enumerate(ground_truth_bb):
        for j, bb_model in enumerate(model_output_bb):
            iou_table[i,j] = bb_intersection_over_union(bb_ground_truth, bb_model)
    
    
    # algorithm to pair the model output bboxes to ground truth bboxes
    # while we have an unassigned gt bbox or there are no more mo bboxes
    #   let u,v be the location of the maximum value in iou_table
    #   create the gt-mo pair (u,v)
    #   zero out the u-th row of iou_table and the v-th column of iou_table
    paired_bbs = [] # ~ a list of tuples of paired bounding boxes
    problematic_bb = []
    gt_bb_inds = np.ones(num_gt_bbox)
    mo_bb_inds = np.ones(num_mo_bbox)
    while np.sum(gt_bb_inds) and np.sum(mo_bb_inds) and iou_table[:].sum():
        u, v = np.unravel_index(iou_table.argmax(), iou_table.shape)
        #print(iou_table)
        #print(u,v)
        #print(gt_bb_inds)
        #print(mo_bb_inds)

        # [u].name != [v].name?
        #Then we found one that has different class names so lets just continue and not add
        #that to the list of paired_bbs
        if dict_data.get(key)[u].get('name') != result_dict_data.get(key)[v].get('name'):
            #good IOU, labels don't match
            mislabeled+=1
            problematic_bb.append((u,v))
            all_problematic_bb.append((u,v))
            gt_bb_inds[u] = 0
            mo_bb_inds[v] = 0
            iou_table[u,:] = 0.0
            iou_table[:,v] = 0.0

            #Need to capture the ground truth image once
            images_to_print[key].append(result_dict_data.get(key)[v])
            groundTruth_mage[key].append(dict_data.get(key)[u])

            continue

        if dict_data.get(key)[u].get('name') == result_dict_data.get(key)[v].get('name') and iou_table[u,v] < 0.5:
            #bad IOU, labels match
            bad_iou+=1
            problematic_bb.append((u,v))
            all_problematic_bb.append((u,v))
            gt_bb_inds[u] = 0
            mo_bb_inds[v] = 0
            iou_table[u,:] = 0.0
            iou_table[:,v] = 0.0

            #Need to capture the ground truth image once
            images_to_print[key].append(result_dict_data.get(key)[v])
            groundTruth_mage[key].append(dict_data.get(key)[u])

            continue

       
        gt_bb_inds[u] = 0
        mo_bb_inds[v] = 0
        iou_table[u,:] = 0.0
        iou_table[:,v] = 0.0

        accurate_predictions[key].append(result_dict_data.get(key)[v])
        accurate_ground_truth[key].append(dict_data.get(key)[u])


    for i in gt_bb_inds:
        if i == 1:
            unpaired+=1
            problematic_bb.append(i)
            all_problematic_bb.append((u,v))
    for j in mo_bb_inds:
        if j == 1:
            unpaired+=1
            problematic_bb.append(j)  
            all_problematic_bb.append((u,v))

In [4]:
print("Statistics: ")
print("Number of total bb's in ground truth set: ", total_bb)
print("Number of problematic bb's: ", len(all_problematic_bb))
print("/*%* of problematic bb's: ", ((len(all_problematic_bb))/(total_bb)*100))
print("Number of mislabeled: ", mislabeled)
print("Correctly labeled but low IOU: ", bad_iou)
print("Unpaired boxes: ", unpaired)

Statistics: 
Number of total bb's in ground truth set:  1954
Number of problematic bb's:  11960
/*%* of problematic bb's:  612.0777891504606
Number of mislabeled:  184
Correctly labeled but low IOU:  199
Unpaired boxes:  11577


In [5]:
show_images(groundTruth_mage,images_to_print)

array([[[211, 201, 194],
        [211, 201, 194],
        [211, 201, 194],
        ...,
        [202, 193, 189],
        [202, 193, 189],
        [202, 193, 189]],

       [[211, 201, 194],
        [211, 201, 194],
        [211, 201, 194],
        ...,
        [202, 193, 189],
        [202, 193, 189],
        [202, 193, 189]],

       [[211, 201, 194],
        [211, 201, 194],
        [211, 201, 194],
        ...,
        [202, 193, 189],
        [202, 193, 189],
        [202, 193, 189]],

       ...,

       [[104, 110, 115],
        [104, 110, 115],
        [104, 110, 115],
        ...,
        [154, 150, 149],
        [154, 150, 149],
        [153, 149, 148]],

       [[105, 111, 116],
        [105, 111, 116],
        [105, 111, 116],
        ...,
        [156, 152, 151],
        [156, 152, 151],
        [155, 151, 150]],

       [[105, 111, 116],
        [105, 111, 116],
        [105, 111, 116],
        ...,
        [160, 156, 155],
        [160, 156, 155],
        [159, 155, 154]]

array([[[229, 213, 200],
        [229, 213, 200],
        [229, 213, 200],
        ...,
        [ 82,  78,  83],
        [ 54,  50,  55],
        [ 53,  46,  53]],

       [[229, 213, 200],
        [229, 213, 200],
        [229, 213, 200],
        ...,
        [ 74,  73,  77],
        [ 52,  51,  55],
        [ 51,  50,  54]],

       [[229, 213, 200],
        [229, 213, 200],
        [229, 213, 200],
        ...,
        [ 70,  72,  73],
        [ 54,  56,  57],
        [ 45,  47,  48]],

       ...,

       [[ 91,  94,  98],
        [ 98, 101, 105],
        [101, 104, 108],
        ...,
        [119, 132, 146],
        [121, 134, 148],
        [123, 136, 150]],

       [[112, 115, 119],
        [117, 120, 124],
        [117, 120, 124],
        ...,
        [119, 132, 146],
        [120, 133, 147],
        [121, 134, 148]],

       [[105, 108, 113],
        [106, 109, 113],
        [107, 110, 114],
        ...,
        [120, 133, 147],
        [121, 134, 148],
        [122, 135, 149]]

array([[[201, 205, 206],
        [201, 205, 206],
        [200, 205, 206],
        ...,
        [ 26,  50,  62],
        [ 19,  43,  55],
        [ 28,  52,  64]],

       [[201, 205, 206],
        [201, 205, 206],
        [200, 205, 206],
        ...,
        [ 19,  44,  54],
        [ 26,  51,  61],
        [ 24,  49,  59]],

       [[203, 205, 206],
        [201, 205, 206],
        [201, 204, 208],
        ...,
        [ 23,  48,  58],
        [ 20,  45,  55],
        [ 21,  46,  56]],

       ...,

       [[124, 150, 167],
        [126, 152, 169],
        [120, 146, 163],
        ...,
        [119, 140, 167],
        [116, 137, 164],
        [120, 141, 168]],

       [[125, 151, 168],
        [118, 144, 161],
        [114, 140, 157],
        ...,
        [123, 144, 171],
        [119, 140, 167],
        [117, 138, 165]],

       [[115, 139, 157],
        [109, 133, 153],
        [121, 146, 166],
        ...,
        [134, 155, 182],
        [130, 151, 178],
        [123, 144, 171]]

array([[[214, 216, 217],
        [214, 216, 217],
        [214, 216, 217],
        ...,
        [196, 175, 153],
        [196, 175, 153],
        [196, 175, 153]],

       [[214, 216, 217],
        [214, 216, 217],
        [214, 216, 217],
        ...,
        [196, 175, 153],
        [196, 175, 153],
        [196, 175, 153]],

       [[213, 215, 216],
        [213, 215, 216],
        [213, 215, 216],
        ...,
        [196, 175, 154],
        [196, 175, 154],
        [196, 175, 154]],

       ...,

       [[ 84,  89,  90],
        [ 82,  87,  88],
        [ 82,  87,  88],
        ...,
        [106, 109, 117],
        [104, 107, 115],
        [ 97, 100, 108]],

       [[ 83,  88,  89],
        [ 82,  87,  88],
        [ 84,  89,  90],
        ...,
        [ 97, 101, 106],
        [102, 106, 111],
        [ 96, 102, 107]],

       [[ 83,  87,  88],
        [ 86,  90,  91],
        [ 90,  94,  95],
        ...,
        [ 94, 100, 105],
        [ 97, 103, 108],
        [ 90,  96, 101]]

array([[[200, 188, 182],
        [199, 189, 182],
        [199, 189, 182],
        ...,
        [210, 198, 192],
        [210, 198, 192],
        [210, 198, 192]],

       [[199, 189, 182],
        [199, 189, 182],
        [199, 189, 182],
        ...,
        [210, 198, 192],
        [210, 198, 192],
        [210, 198, 192]],

       [[199, 189, 182],
        [199, 189, 182],
        [199, 189, 182],
        ...,
        [210, 198, 192],
        [210, 198, 192],
        [210, 198, 192]],

       ...,

       [[142, 153, 161],
        [112, 123, 131],
        [116, 127, 135],
        ...,
        [102, 107, 116],
        [101, 106, 115],
        [106, 111, 120]],

       [[107, 118, 126],
        [107, 118, 126],
        [109, 120, 128],
        ...,
        [ 92,  97, 106],
        [ 93,  98, 107],
        [ 99, 104, 113]],

       [[111, 121, 131],
        [104, 115, 123],
        [103, 114, 122],
        ...,
        [104, 109, 118],
        [112, 117, 126],
        [117, 122, 131]]

...

array([[[199, 203, 204],
        [199, 203, 204],
        [199, 203, 204],
        ...,
        [ 91, 117, 153],
        [ 92, 118, 154],
        [ 91, 119, 154]],

       [[199, 203, 204],
        [199, 203, 204],
        [199, 203, 204],
        ...,
        [ 90, 116, 152],
        [ 89, 117, 152],
        [ 90, 118, 153]],

       [[199, 203, 204],
        [199, 203, 204],
        [199, 203, 204],
        ...,
        [ 87, 115, 150],
        [ 88, 116, 151],
        [ 88, 116, 151]],

       ...,

       [[131, 157, 181],
        [136, 162, 186],
        [146, 172, 196],
        ...,
        [141, 170, 197],
        [148, 177, 204],
        [152, 181, 208]],

       [[125, 151, 175],
        [132, 158, 182],
        [137, 163, 187],
        ...,
        [131, 160, 187],
        [133, 162, 189],
        [141, 170, 197]],

       [[122, 148, 172],
        [121, 147, 171],
        [127, 153, 177],
        ...,
        [133, 162, 189],
        [126, 155, 182],
        [129, 158, 185]]

array([[[119, 148, 175],
        [122, 151, 178],
        [122, 151, 178],
        ...,
        [ 75,  81,  80],
        [ 69,  74,  73],
        [ 45,  50,  49]],

       [[ 86, 115, 142],
        [ 81, 110, 137],
        [ 79, 108, 135],
        ...,
        [ 55,  66,  64],
        [ 39,  50,  48],
        [ 67,  78,  76]],

       [[ 77, 106, 133],
        [ 79, 108, 135],
        [ 77, 106, 133],
        ...,
        [ 57,  76,  73],
        [ 95, 112, 109],
        [134, 151, 148]],

       ...,

       [[118, 124, 131],
        [126, 132, 139],
        [131, 137, 144],
        ...,
        [128, 134, 139],
        [128, 134, 139],
        [131, 137, 142]],

       [[117, 123, 128],
        [128, 134, 139],
        [132, 138, 143],
        ...,
        [128, 134, 139],
        [123, 129, 134],
        [127, 133, 138]],

       [[122, 128, 133],
        [129, 135, 140],
        [130, 136, 141],
        ...,
        [128, 134, 139],
        [128, 134, 139],
        [128, 134, 139]]

array([[[146, 171, 191],
        [146, 171, 191],
        [146, 171, 191],
        ...,
        [238, 226, 224],
        [238, 226, 224],
        [238, 226, 224]],

       [[146, 171, 191],
        [146, 171, 191],
        [146, 171, 191],
        ...,
        [238, 226, 224],
        [238, 226, 224],
        [236, 227, 224]],

       [[146, 171, 191],
        [146, 171, 191],
        [146, 171, 191],
        ...,
        [238, 226, 224],
        [236, 227, 224],
        [236, 227, 224]],

       ...,

       [[ 87,  99, 109],
        [ 88, 100, 110],
        [ 87,  99, 109],
        ...,
        [102, 115, 129],
        [102, 115, 129],
        [102, 115, 129]],

       [[ 81,  93, 103],
        [ 86,  98, 108],
        [ 85,  97, 107],
        ...,
        [102, 115, 129],
        [102, 115, 129],
        [103, 116, 130]],

       [[ 80,  92, 102],
        [ 87,  99, 109],
        [ 86,  98, 108],
        ...,
        [103, 116, 130],
        [103, 116, 130],
        [103, 116, 130]]

array([[[211, 198, 190],
        [211, 198, 190],
        [211, 198, 190],
        ...,
        [220, 218, 217],
        [220, 218, 217],
        [220, 218, 217]],

       [[209, 198, 190],
        [209, 198, 190],
        [209, 197, 191],
        ...,
        [220, 218, 217],
        [220, 218, 217],
        [220, 218, 217]],

       [[208, 198, 191],
        [208, 198, 191],
        [208, 198, 191],
        ...,
        [220, 218, 217],
        [220, 218, 217],
        [220, 218, 217]],

       ...,

       [[ 92,  96,  91],
        [ 93,  97,  92],
        [ 94,  98,  93],
        ...,
        [138, 148, 158],
        [130, 142, 152],
        [153, 165, 175]],

       [[ 93,  97,  92],
        [ 94,  98,  93],
        [ 93,  97,  92],
        ...,
        [119, 129, 139],
        [113, 125, 135],
        [107, 119, 129]],

       [[ 93,  97,  92],
        [ 93,  97,  92],
        [ 90,  94,  89],
        ...,
        [160, 170, 180],
        [171, 183, 193],
        [149, 161, 171]]

array([[[117, 153, 193],
        [116, 152, 192],
        [116, 152, 192],
        ...,
        [221, 216, 215],
        [221, 216, 215],
        [221, 216, 215]],

       [[118, 154, 194],
        [117, 153, 193],
        [117, 153, 193],
        ...,
        [221, 216, 215],
        [221, 216, 215],
        [221, 216, 215]],

       [[118, 154, 194],
        [118, 154, 194],
        [118, 154, 194],
        ...,
        [221, 216, 215],
        [221, 216, 215],
        [221, 216, 215]],

       ...,

       [[128, 139, 147],
        [133, 144, 152],
        [133, 142, 151],
        ...,
        [133, 138, 147],
        [129, 134, 143],
        [127, 132, 141]],

       [[134, 145, 153],
        [134, 145, 153],
        [132, 141, 150],
        ...,
        [131, 136, 145],
        [132, 137, 146],
        [131, 136, 145]],

       [[133, 144, 152],
        [133, 144, 152],
        [130, 139, 148],
        ...,
        [129, 134, 143],
        [136, 141, 150],
        [131, 136, 145]]